In [1]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

import scipy.signal
import holidays

import tensorflow as tf

from pandas.tseries.offsets import MonthEnd

import seaborn as sb

import IPython

# Aux Functions for Solcast and PV data processing

def get_solcast(path):
    file = pd.read_csv(path)
    file.index = pd.to_datetime(file['PeriodEnd'])
    file = file.resample('15T').pad()
    
    return file


def get_pv(path):
    file = pd.read_csv(path)
    file.index = pd.to_datetime(file['datetime_utc'])
    file = file.resample('15T').mean()
    
    return file


def get_solcastPV(df1, df2):
    '''
    df1: PV dataframe
    df2: Solcast dataframe
    '''
    
    # Filter both dataframes for 2019 and 2020
    try:
        temp_df1 = df1['2019':'2021-04-01']
        temp_df2 = df2['2019':'2021-04-01']
        
        # Check if data is complete. If not, match the smaller indexes
        if temp_df2.shape[0] < temp_df1.shape[0]:
            last_entry = temp_df2.index
            temp_df1 = temp_df1['2019':'{}'.format(temp_df2.index[-1].tz_convert(None))]


        # Only considering 2019 and 2020 since data is complete for that period
        temp_data = pd.DataFrame({'PV': temp_df1['pv'].values}, index=temp_df1.index)
        for i in np.arange(3, len(temp_df2.columns)):
            temp_data[temp_df2.columns[i]] = temp_df2[temp_df2.columns[i]].shift(-1).values
            
        return temp_data
    except:
        temp_df1 = df1['2019':'2020']
        temp_df2 = df2['2019':'2020']
        
        # Check if data is complete. If not, match the smaller indexes
        if temp_df2.shape[0] < temp_df1.shape[0]:
            last_entry = temp_df2.index
            temp_df1 = temp_df1['2019':'{}'.format(temp_df2.index[-1].tz_convert(None))]


        # Only considering 2019 and 2020 since data is complete for that period
        temp_data = pd.DataFrame({'PV': temp_df1['pv'].values}, index=temp_df1.index)
        for i in np.arange(3, len(temp_df2.columns)):
            temp_data[temp_df2.columns[i]] = temp_df2[temp_df2.columns[i]].shift(-1).values
            
        return temp_data

# Get data and build a dictionary for preprocessing

data = {}

folders = glob.glob('C:/Users/FEEL/Jupyter/ecgomes/upacs_study/data/*')
for folder in folders:
    # Load each of the files inside the folder
    temp_pv = get_pv('{}/pv.csv'.format(folder))
    temp_solcast = get_solcast('{}/solcast.csv'.format(folder))
    
    # Join the files into a single dataframe
    temp_upac = get_solcastPV(temp_pv, temp_solcast)
    
    temp_name = folder.split('\\')[1]
    data[temp_name] = temp_upac
    
    print('{} date range: {} - {}'.format(temp_name, temp_upac.index[0], temp_upac.index[-1]))

In [30]:
# Get the data from the UK dataset

data = {}

temp_data = pd.read_csv('UK/train/pv_train_set4.csv')
temp_data.index = pd.to_datetime(temp_data['datetime'])
temp_data.drop('datetime', axis=1, inplace=True)
temp_data = temp_data.resample('15T').pad()

data['uk'] = temp_data

In [31]:
# Aux Functions for adding 2D time information

import datetime

def days_2d(df):
    '''
    Adds 2D time information for single days
    df: dataframe to add the information
    '''
    # Map the index into seconds
    timestamp_s = pd.to_datetime(df.index.values).map(datetime.datetime.timestamp)
    
    # Since we're calculating the cos and sin values from seconds, it's 60 seconds into 60 min into 24 hours per day
    day_calc = 24*60*60
    
    # Calculate the values
    dayx = np.cos((2*np.pi/day_calc) * timestamp_s)
    dayy = np.sin((2*np.pi/day_calc) * timestamp_s)
    
    return dayx, dayy
    

def years_2d(df):
    '''
    Adds 2D time representation throught a year
    df: dataframe to add the information
    '''
    # Add Year Information

    day_year = df.index.dayofyear
    year_constant = 365.2524

    yearx = np.cos((2*np.pi/year_constant) * day_year)
    yeary = np.sin((2*np.pi/year_constant) * day_year)
    
    return yearx, yeary

In [32]:
# Add the 2D time information to the data

for upac in data.keys():
    dayx, dayy = days_2d(data[upac])
    yearx, yeary = years_2d(data[upac])
    
    data[upac]['Day X'] = dayx
    data[upac]['Day Y'] = dayy
    
    data[upac]['Year X'] = yearx
    data[upac]['Year Y'] = yeary

In [56]:
# Split the data for training, validation and testing

data_train = {}
data_val = {}
data_test = {}

for upac in data.keys():
    data_train[upac] = data[upac]['2018']
    data_val[upac] = data[upac]['2019-01':'2019-03']
    data_test[upac] = data[upac]['2019-04':]

C:\Users\FEEL\AppData\Local\Temp/ipykernel_14944/4210865765.py:8: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  data_train[upac] = data[upac]['2018']


In [57]:
# Aux Function for filtering data

def filter_by_points(df, frequency='D', num_points=1440, return_dictionary=False):
    
    df_dropped = df.dropna()
    grouper = df_dropped.groupby(pd.Grouper(freq=frequency))
    
    output = 0
    if return_dictionary:
        new_dict = {}
        for i in grouper:
            if (len(i[1]) != num_points):
                pass
            else:
                new_dict[i[0]] = pd.DataFrame(i[1])
        output = new_dict
    else:
        new_df = pd.DataFrame({})
        for i in grouper:
            if (len(i[1]) != num_points):
                pass
            else:
                new_df = new_df.append(pd.DataFrame(i[1]))
        output = new_df
            
    return output

In [58]:
# Filter the data by number of points that should be present in a single day

filtered_train = {}
filtered_val = {}
filtered_test = {}

for upac in data_train.keys():
    filtered_train[upac] = filter_by_points(data_train[upac], frequency='D', num_points=1440/15)
    filtered_val[upac] = filter_by_points(data_val[upac], frequency='D', num_points=1440/15)
    filtered_test[upac] = filter_by_points(data_test[upac], frequency='D', num_points=1440/15)

In [59]:
# Select columns to use

USED_COLUMNS = ['irradiance_Wm-2', 
                'pv_power_mw', 
                'panel_temp_C', 
                'Day Y', 'Day X',
                'Year Y', 'Year X']

In [60]:
# Data Normalization
# We don't want to normalize PV so we can capture diferences more easily

# Feature range
# PV - greater than 0
# Irradiance - normalize between 0 and 1
# Panel Temp - Unchanged
# Day X, Y and Year X and Y - already between -1 and 1

MAX_GHI = 1117.440000 # max value on the training set

normalized_train = {}
normalized_val = {}
normalized_test = {}

for upac in filtered_train.keys():
    normalized_train[upac] = filtered_train[upac][USED_COLUMNS].copy(deep=True)
    normalized_val[upac] = filtered_val[upac][USED_COLUMNS].copy(deep=True)
    normalized_test[upac] = filtered_test[upac][USED_COLUMNS].copy(deep=True)
    
    normalized_train[upac]['irradiance_Wm-2'] = normalized_train[upac]['irradiance_Wm-2'] / MAX_GHI
    normalized_val[upac]['irradiance_Wm-2'] = normalized_val[upac]['irradiance_Wm-2'] / MAX_GHI
    normalized_test[upac]['irradiance_Wm-2'] = normalized_test[upac]['irradiance_Wm-2'] / MAX_GHI

In [61]:
# Aux Function DNN Specific - Data Reshaping and Windowing

def split_sequence(sequence, n_steps, n_intervals):
    X = list()
    for i in np.arange(0, len(sequence), n_intervals):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x = sequence[i:end_ix]
        X.append(seq_x)
    return np.array(X)

def split_dataframe(df, input_width):
    current_list = []
    for i in df.columns[1:]:
        current_sequence = split_sequence(df[i], input_width, 1)
        current_list.append(current_sequence)
        
    stacked_list = np.stack(current_list, axis=-1)
    
    return stacked_list


def do_windowing(df, input_width=1, y_col='pv_power_mw'):
    '''
    Perform the windowing on the dataframe
    df: input dataframe to perform windowing
    input_width: temporal dimension size or number of timesteps
    y_col: name of column for the prediction
    '''
    
    temp_x = split_dataframe(df, input_width)
    temp_y = df[y_col].values[input_width-1:-1]
    
    return temp_x, temp_y

In [62]:
# Do windowing on the data

X_train = {}
y_train = {}

X_val = {}
y_val = {}

X_test = {}
y_test = {}

for upac in normalized_train.keys():
    trainx = normalized_train[upac].drop('pv_power_mw', axis=1)
    trainy = normalized_train[upac]['pv_power_mw']
    valx = normalized_val[upac].drop('pv_power_mw', axis=1)
    valy = normalized_val[upac]['pv_power_mw']
    testx = normalized_test[upac].drop('pv_power_mw', axis=1)
    testy = normalized_test[upac]['pv_power_mw']
    
    X_train[upac] = trainx
    X_val[upac] = valx
    X_test[upac] = testx
    
    y_train[upac] = trainy
    y_val[upac] = valy
    y_test[upac] = testy

In [63]:
# Aux Functions to compile the model and add necessary info

def compile_and_fit(model, train_x, train_y, val_x, val_y,
                    max_epochs=1000, es_patience=200, rlr_patience=100, 
                    adam_lr=0.001):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=es_patience,
                                                      mode='min')
    
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                     patience=rlr_patience)

    model.compile(loss=tf.keras.losses.mean_squared_error,
                  optimizer=tf.optimizers.Adam(learning_rate=adam_lr),
                  metrics=[tf.metrics.MeanAbsoluteError(name='mae')])

    history = model.fit(train_x, train_y,
                        batch_size=32,
                        epochs=max_epochs,
                        validation_data=(val_x, val_y),
                        callbacks=[early_stopping, reduce_lr])
    return history

In [64]:
# Aux Function for predicting and storing values

def do_predictions(dictionary, save_path, X, y, index):
    # Go through each model in the dictionary
    for model in dictionary.keys():
        print('Doing {}'.format(model))
        
        temp_path = '{}/{}.csv'.format(save_path, model)
        
        y_pred = dictionary[model].predict(X, verbose=1)
        y_pred = pd.DataFrame(y_pred[:, :], columns=['pv_power_mw'],
                              index=index)
        
        y_pred.to_csv(temp_path)
        
    # Also save ground-truth data at the end of the loop
    y_true = pd.DataFrame(y, columns=['pv_power_mw'],
                          index=index)
    
    temp_path_gt = '{}/gt.csv'.format(save_path)
    y_true.to_csv(temp_path_gt)
    
    
def predict_upacs(model_dictionary, upac_name, X_train, y_train, X_val, y_val, X_test, y_test):
    # Simply call the function above for each of the settings to simplify
    
    print('Doing training for {}'.format(upac_name))
    temp_path = 'results/tabnet/{}_all/train'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_train[upac_name], 
                   y=y_train[upac_name],
                   index=normalized_train[upac_name].index)
    IPython.display.clear_output()
    
    print('Doing validation for {}'.format(upac_name))
    temp_path = 'results/tabnet/{}_all/val'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_val[upac_name], 
                   y=y_val[upac_name],
                   index=normalized_val[upac_name].index)
    IPython.display.clear_output()
    
    print('Doing testing for {}'.format(upac_name))
    temp_path = 'results/tabnet/{}_all/test'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_test[upac_name], 
                   y=y_test[upac_name],
                   index=normalized_test[upac_name].index)
    IPython.display.clear_output()

In [51]:
# Import the TabNet implementation

# From: https://github.com/titu1994/tf-TabNet
from tabnet import StackedTabNetRegressor

In [65]:
# Optuna for TabNet hyperparameter optimization

import optuna
import sklearn

import random

random.seed(69)
np.random.seed(69)
tf.random.set_seed(69)

def optuna_create_model(trial):
    # Do search for n_estimators, max_depth, reg_alpha and reg_lambda
    #sug_layers = trial.suggest_int('num_layers', 1, 4)
    sug_fdim = trial.suggest_int('feature_dim', 32, 72)
    sug_odim = trial.suggest_int('output_dim', 4, 12)
    sug_dsteps = trial.suggest_int('num_decision_steps', 1, 4)
    
    sug_model = StackedTabNetRegressor(feature_columns=None,
                                       num_layers=1, #sug_layers,
                                       num_regressors=1, 
                                       feature_dim=sug_fdim,
                                       num_features=6, 
                                       output_dim=sug_odim,
                                       num_decision_steps=sug_dsteps,
                                       num_groups=1)
    
    return sug_model


def optuna_create_training(model):
    compile_and_fit(model, X_train['uk'], y_train['uk'],
                    X_val['uk'], y_val['uk'],
                    max_epochs=50, rlr_patience=10, es_patience=20)
    
    
def optuna_create_evaluation(model):
    temp_yhat = model.predict(X_val['uk'])
    return sklearn.metrics.mean_squared_error(y_val['uk'], temp_yhat)
    
    
def objective(trial):
    # Instantiate the model
    temp_model = optuna_create_model(trial)
    
    # Train the model
    optuna_create_training(temp_model)
    
    # Evaluate model
    metrics_val = optuna_create_evaluation(temp_model)
    
    return metrics_val

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2022-08-18 02:56:57,794] A new study created in memory with name: no-name-e01230c0-5761-43fd-affc-0a6d516d0b50
c:\users\feel\jupyter\ecgomes\upacs_study\venv\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

[TabNet]: 29 features will be used for decision steps.
Epoch 1/50
1086/1086 [==============================] - 5s 3ms/step - loss: 0.0446 - mae: 0.0919 - val_loss: 0.0146 - val_mae: 0.0488 - lr: 0.0010
Epoch 2/50
1086/1086 [==============================] - 3s 3ms/step - loss: 0.0243 - mae: 0.0678 - val_loss: 0.0143 - val_mae: 0.0588 - lr: 0.0010
Epoch 3/50
1086/1086 [==============================] - 3s 3ms/step - loss: 0.0224 - mae: 0.0648 - val_loss: 0.0136 - val_mae: 0.0492 - lr: 0.0010
Epoch 4/50
1086/1086 [==============================] - 3s 3ms/step - loss: 0.0221 - mae: 0.0649 - val_loss: 0.0160 - val_mae: 0.0557 - lr: 0.0010
Epoch 5/50
1086/1086 [==============================] - 3s 3ms/step - loss: 0.0212 - mae: 0.0608 - val_loss: 0.0268 - val_mae: 0.0778 - lr: 0.0010
Epoch 6/50
1086/1086 [==============================] - 3s 3ms/step - loss: 0.0202 - mae: 0.0563 - val_loss: 0.0205 - val_mae: 0.0616 - lr: 0.0010
Epoch 7/50
1086/1086 [==============================] - 3s 3ms/

1086/1086 [==============================] - 1s 1ms/step - loss: 0.0125 - mae: 0.0293 - val_loss: 0.0122 - val_mae: 0.0358 - lr: 1.0000e-04
Epoch 32/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0124 - mae: 0.0293 - val_loss: 0.0127 - val_mae: 0.0381 - lr: 1.0000e-04
Epoch 33/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0125 - mae: 0.0293 - val_loss: 0.0134 - val_mae: 0.0380 - lr: 1.0000e-04
Epoch 34/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0124 - mae: 0.0293 - val_loss: 0.0145 - val_mae: 0.0413 - lr: 1.0000e-04
Epoch 35/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0124 - mae: 0.0296 - val_loss: 0.0125 - val_mae: 0.0358 - lr: 1.0000e-04
Epoch 36/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0124 - mae: 0.0293 - val_loss: 0.0119 - val_mae: 0.0343 - lr: 1.0000e-04
[I 2022-08-18 02:58:57,369] Trial 1 finished with value: 0.011868367301587723 and parameters: {'feat

Epoch 19/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0131 - mae: 0.0313 - val_loss: 0.0127 - val_mae: 0.0367 - lr: 1.0000e-04
Epoch 20/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0131 - mae: 0.0309 - val_loss: 0.0117 - val_mae: 0.0349 - lr: 1.0000e-04
Epoch 21/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0131 - mae: 0.0312 - val_loss: 0.0127 - val_mae: 0.0370 - lr: 1.0000e-04
Epoch 22/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0131 - mae: 0.0309 - val_loss: 0.0130 - val_mae: 0.0380 - lr: 1.0000e-04
Epoch 23/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0131 - mae: 0.0311 - val_loss: 0.0138 - val_mae: 0.0397 - lr: 1.0000e-04
Epoch 24/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0131 - mae: 0.0311 - val_loss: 0.0130 - val_mae: 0.0375 - lr: 1.0000e-04
[I 2022-08-18 03:01:22,275] Trial 3 finished with value: 0.013024228234663772 and parame

...

Epoch 8/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0185 - mae: 0.0498 - val_loss: 0.0116 - val_mae: 0.0384 - lr: 0.0010
Epoch 9/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0176 - mae: 0.0485 - val_loss: 0.0248 - val_mae: 0.0640 - lr: 0.0010
Epoch 10/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0173 - mae: 0.0468 - val_loss: 0.0135 - val_mae: 0.0401 - lr: 0.0010
Epoch 11/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0174 - mae: 0.0468 - val_loss: 0.0107 - val_mae: 0.0363 - lr: 0.0010
Epoch 12/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0170 - mae: 0.0445 - val_loss: 0.0185 - val_mae: 0.0489 - lr: 0.0010
Epoch 13/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0164 - mae: 0.0437 - val_loss: 0.0133 - val_mae: 0.0412 - lr: 0.0010
Epoch 14/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0167 - mae: 0.0442 - val_loss: 0.01

1086/1086 [==============================] - 1s 1ms/step - loss: 0.0127 - mae: 0.0309 - val_loss: 0.0129 - val_mae: 0.0371 - lr: 1.0000e-04
Epoch 36/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0127 - mae: 0.0306 - val_loss: 0.0120 - val_mae: 0.0351 - lr: 1.0000e-04
[I 2022-08-18 04:28:28,050] Trial 94 finished with value: 0.01198091639303665 and parameters: {'feature_dim': 60, 'output_dim': 6, 'num_decision_steps': 1}. Best is trial 54 with value: 0.011344388322400605.
Epoch 1/50
1086/1086 [==============================] - 2s 1ms/step - loss: 0.1237 - mae: 0.1650 - val_loss: 0.0168 - val_mae: 0.0595 - lr: 0.0010
Epoch 2/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0277 - mae: 0.0749 - val_loss: 0.0134 - val_mae: 0.0475 - lr: 0.0010
Epoch 3/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0244 - mae: 0.0671 - val_loss: 0.0125 - val_mae: 0.0434 - lr: 0.0010
Epoch 4/50
1086/1086 [==============================] - 1s 

1086/1086 [==============================] - 2s 1ms/step - loss: 0.0129 - mae: 0.0311 - val_loss: 0.0116 - val_mae: 0.0355 - lr: 1.0000e-04
Epoch 26/50
1086/1086 [==============================] - 2s 1ms/step - loss: 0.0128 - mae: 0.0307 - val_loss: 0.0132 - val_mae: 0.0385 - lr: 1.0000e-04
Epoch 27/50
1086/1086 [==============================] - 2s 1ms/step - loss: 0.0125 - mae: 0.0292 - val_loss: 0.0126 - val_mae: 0.0374 - lr: 1.0000e-05
Epoch 28/50
1086/1086 [==============================] - 2s 1ms/step - loss: 0.0125 - mae: 0.0290 - val_loss: 0.0126 - val_mae: 0.0372 - lr: 1.0000e-05
Epoch 29/50
1086/1086 [==============================] - 2s 1ms/step - loss: 0.0125 - mae: 0.0289 - val_loss: 0.0129 - val_mae: 0.0379 - lr: 1.0000e-05
Epoch 30/50
1086/1086 [==============================] - 2s 1ms/step - loss: 0.0125 - mae: 0.0289 - val_loss: 0.0126 - val_mae: 0.0372 - lr: 1.0000e-05
Epoch 31/50
1086/1086 [==============================] - 2s 1ms/step - loss: 0.0125 - mae: 0.0289 - 

Epoch 3/50
1086/1086 [==============================] - 2s 1ms/step - loss: 0.0241 - mae: 0.0645 - val_loss: 0.0124 - val_mae: 0.0431 - lr: 0.0010
Epoch 4/50
1086/1086 [==============================] - 2s 1ms/step - loss: 0.0230 - mae: 0.0625 - val_loss: 0.0140 - val_mae: 0.0487 - lr: 0.0010
Epoch 5/50
1086/1086 [==============================] - 2s 1ms/step - loss: 0.0209 - mae: 0.0562 - val_loss: 0.0270 - val_mae: 0.0715 - lr: 0.0010
Epoch 6/50
1086/1086 [==============================] - 2s 1ms/step - loss: 0.0202 - mae: 0.0545 - val_loss: 0.0111 - val_mae: 0.0342 - lr: 0.0010
Epoch 7/50
1086/1086 [==============================] - 2s 1ms/step - loss: 0.0191 - mae: 0.0516 - val_loss: 0.0151 - val_mae: 0.0501 - lr: 0.0010
Epoch 8/50
1086/1086 [==============================] - 2s 1ms/step - loss: 0.0193 - mae: 0.0528 - val_loss: 0.0135 - val_mae: 0.0459 - lr: 0.0010
Epoch 9/50
1086/1086 [==============================] - 2s 1ms/step - loss: 0.0182 - mae: 0.0507 - val_loss: 0.0315 - 

1086/1086 [==============================] - 1s 1ms/step - loss: 1.3999 - mae: 0.6041 - val_loss: 0.9063 - val_mae: 0.4320 - lr: 1.0000e-04
Epoch 21/50
1086/1086 [==============================] - 1s 1ms/step - loss: 1.3999 - mae: 0.6041 - val_loss: 0.9063 - val_mae: 0.4320 - lr: 1.0000e-04
[I 2022-08-18 04:32:10,433] Trial 99 finished with value: 0.9063368534482759 and parameters: {'feature_dim': 61, 'output_dim': 5, 'num_decision_steps': 1}. Best is trial 54 with value: 0.011344388322400605.


In [66]:
# TabNet best params (all fetures)

study.best_params

{'feature_dim': 63, 'output_dim': 7, 'num_decision_steps': 1}

In [67]:
# Train 10 Models for UK - All

N_MODELS = 10
uk_models_tn = {}

for i in np.arange(N_MODELS):
    IPython.display.clear_output()
    print('Current Model: {:02d}'.format(i+1))
    
    tabnet_regressor = StackedTabNetRegressor(feature_columns=None,
                                              num_layers=1, #study.best_params['num_layers'],
                                              num_regressors=1, 
                                              feature_dim=study.best_params['feature_dim'],
                                              num_features=6, 
                                              output_dim=study.best_params['output_dim'],
                                              num_decision_steps=study.best_params['num_decision_steps'],
                                              num_groups=1)
    current_net_history = compile_and_fit(tabnet_regressor, 
                                          X_train['uk'], y_train['uk'],
                                          X_val['uk'], y_val['uk'])
    
    tabnet_regressor.save('models/tabnet/uk_all/Model {:02d}'.format(i+1), save_format='tf')
    uk_models_tn['Model {:02d}'.format(i+1)] = tabnet_regressor

Current Model: 10
Epoch 1/1000
1086/1086 [==============================] - 2s 1ms/step - loss: 0.1816 - mae: 0.1985 - val_loss: 0.0150 - val_mae: 0.0553 - lr: 0.0010
Epoch 2/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0264 - mae: 0.0717 - val_loss: 0.0150 - val_mae: 0.0554 - lr: 0.0010
Epoch 3/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0233 - mae: 0.0640 - val_loss: 0.0115 - val_mae: 0.0413 - lr: 0.0010
Epoch 4/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0211 - mae: 0.0587 - val_loss: 0.0114 - val_mae: 0.0413 - lr: 0.0010
Epoch 5/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0200 - mae: 0.0545 - val_loss: 0.0286 - val_mae: 0.0723 - lr: 0.0010
Epoch 6/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0194 - mae: 0.0530 - val_loss: 0.0106 - val_mae: 0.0325 - lr: 0.0010
Epoch 7/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0187 - m

1086/1086 [==============================] - 1s 1ms/step - loss: 0.0137 - mae: 0.0365 - val_loss: 0.0146 - val_mae: 0.0439 - lr: 0.0010
Epoch 56/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0136 - mae: 0.0367 - val_loss: 0.0216 - val_mae: 0.0519 - lr: 0.0010
Epoch 57/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0136 - mae: 0.0372 - val_loss: 0.0131 - val_mae: 0.0379 - lr: 0.0010
Epoch 58/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0134 - mae: 0.0360 - val_loss: 0.0147 - val_mae: 0.0429 - lr: 0.0010
Epoch 59/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0137 - mae: 0.0368 - val_loss: 0.0118 - val_mae: 0.0356 - lr: 0.0010
Epoch 60/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0135 - mae: 0.0365 - val_loss: 0.0135 - val_mae: 0.0415 - lr: 0.0010
Epoch 61/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0134 - mae: 0.0363 - val_loss: 0.

Epoch 110/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0134 - mae: 0.0354 - val_loss: 0.0133 - val_mae: 0.0408 - lr: 0.0010
Epoch 111/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0118 - mae: 0.0326 - val_loss: 0.0147 - val_mae: 0.0412 - lr: 0.0010
Epoch 112/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0119 - mae: 0.0340 - val_loss: 0.0121 - val_mae: 0.0354 - lr: 0.0010
Epoch 113/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0117 - mae: 0.0332 - val_loss: 0.0116 - val_mae: 0.0346 - lr: 0.0010
Epoch 114/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0117 - mae: 0.0332 - val_loss: 0.0114 - val_mae: 0.0368 - lr: 0.0010
Epoch 115/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0123 - mae: 0.0343 - val_loss: 0.0128 - val_mae: 0.0382 - lr: 0.0010
Epoch 116/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0113 - mae: 

1086/1086 [==============================] - 1s 1ms/step - loss: 0.0086 - mae: 0.0257 - val_loss: 0.0139 - val_mae: 0.0399 - lr: 1.0000e-04
Epoch 165/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0082 - mae: 0.0255 - val_loss: 0.0129 - val_mae: 0.0372 - lr: 1.0000e-04
Epoch 166/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0081 - mae: 0.0255 - val_loss: 0.0132 - val_mae: 0.0377 - lr: 1.0000e-04
Epoch 167/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0080 - mae: 0.0257 - val_loss: 0.0130 - val_mae: 0.0374 - lr: 1.0000e-04
Epoch 168/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0080 - mae: 0.0255 - val_loss: 0.0130 - val_mae: 0.0374 - lr: 1.0000e-04
Epoch 169/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0079 - mae: 0.0253 - val_loss: 0.0132 - val_mae: 0.0379 - lr: 1.0000e-04
Epoch 170/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.007

Epoch 217/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0066 - mae: 0.0247 - val_loss: 0.0134 - val_mae: 0.0388 - lr: 1.0000e-04
Epoch 218/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0066 - mae: 0.0249 - val_loss: 0.0139 - val_mae: 0.0406 - lr: 1.0000e-04
Epoch 219/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0066 - mae: 0.0248 - val_loss: 0.0129 - val_mae: 0.0376 - lr: 1.0000e-04
Epoch 220/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0065 - mae: 0.0245 - val_loss: 0.0138 - val_mae: 0.0395 - lr: 1.0000e-04
Epoch 221/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0064 - mae: 0.0244 - val_loss: 0.0136 - val_mae: 0.0388 - lr: 1.0000e-04
Epoch 222/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0064 - mae: 0.0244 - val_loss: 0.0137 - val_mae: 0.0402 - lr: 1.0000e-04
Epoch 223/1000
1086/1086 [==============================] - 1s 1ms/ste

In [68]:
# UPAC08 Predictions for all features

predict_upacs(model_dictionary=uk_models_tn, 
              upac_name='uk',
              X_train=X_train, y_train=y_train,
              X_val=X_val, y_val=y_val,
              X_test=X_test, y_test=y_test)